In [14]:
import json
import numpy as np

## 1. Load model

In [ ]:
import argparse
parser = argparse.ArgumentParser()
# Set-up parameters
parser.add_argument('--device', default='cuda', type=str, help='Name of device to use for tensor computations (cuda/cpu)')
parser.add_argument('--display_step', default=10, type=int, help='Interval in batches between display of training metrics')
parser.add_argument('--res_dir', default='./results', type=str)
parser.add_argument('--ex_name', default='debug', type=str)
parser.add_argument('--use_gpu', default=True, type=bool)
parser.add_argument('--gpu', default=0, type=int)
parser.add_argument('--seed', default=111, type=int)

# CATH
# dataset parameters
parser.add_argument('--data_name', default='CATH', choices=['CATH', 'TS50'])
parser.add_argument('--data_root', default='./data/')
parser.add_argument('--batch_size', default=8, type=int)
parser.add_argument('--num_workers', default=8, type=int)

# method parameters
parser.add_argument('--method', default='SimDesign', choices=['SimDesign'])
parser.add_argument('--config_file', '-c', default=None, type=str)
parser.add_argument('--hidden_dim',  default=128, type=int)
parser.add_argument('--node_features',  default=128, type=int)
parser.add_argument('--edge_features',  default=128, type=int)
parser.add_argument('--k_neighbors',  default=30, type=int)
parser.add_argument('--dropout',  default=0.1, type=int)
parser.add_argument('--num_encoder_layers', default=10, type=int)

# Training parameters
parser.add_argument('--epoch', default=100, type=int, help='end epoch')
parser.add_argument('--log_step', default=1, type=int)
parser.add_argument('--lr', default=0.001, type=float, help='Learning rate')
parser.add_argument('--patience', default=100, type=int)

# ProDesign parameters
parser.add_argument('--updating_edges', default=4, type=int)
parser.add_argument('--node_dist', default=1, type=int)
parser.add_argument('--node_angle', default=1, type=int)
parser.add_argument('--node_direct', default=1, type=int)
parser.add_argument('--edge_dist', default=1, type=int)
parser.add_argument('--edge_angle', default=1, type=int)
parser.add_argument('--edge_direct', default=1, type=int)
parser.add_argument('--virtual_num', default=3, type=int)
args = parser.parse_args([])

import torch
from main import Exp
from parser import create_parser
exp = Exp(args)
svpath = './results/ProDesign/'
exp.method.model.load_state_dict(torch.load(svpath+'checkpoint.pth'))

## 2. Results on CATH4.2

In [ ]:
exp.test()
print("median: {:.4f}\t mean: {:.4f}\t std: {:.4f}\t min: {:.4f}\t max: {:.4f}".format(exp.method.median_recovery, exp.method.mean_recovery, exp.method.std_recovery, exp.method.min_recovery, exp.method.max_recovery))

## 3. Results on TS50

In [15]:
from API.dataloader import make_cath_loader
from API.cath_dataset import CATH

with open('./data/ts/ts50.json','r') as f:
    ts50 = json.load(f)


ts50_list = []
for entry in ts50:
    coords = np.array(entry['coords'])
    ts50_list.append(
        {
                    'title':entry['name'],
                    'seq':entry['seq'],
                    'CA':coords[:,1,:],
                    'C':coords[:,2,:],
                    'O':coords[:,3,:],
                    'N':coords[:,0,:]
        }
    )
exp.test_loader = make_cath_loader(CATH(data=ts50_list), 'SimDesign', 8)

In [16]:
exp.test()
print("median: {:.4f}\t mean: {:.4f}\t std: {:.4f}\t min: {:.4f}\t max: {:.4f}".format(exp.method.median_recovery, exp.method.mean_recovery, exp.method.std_recovery, exp.method.min_recovery, exp.method.max_recovery))

100%|██████████| 50/50 [00:01<00:00, 29.85it/s]


Test Perp: 3.8553, Test Rec: 0.5872

Category Unknown Rec: 0.5872

median: 0.5872	 mean: 0.5654	 std: 0.0887	 min: 0.3724	 max: 0.7530


## Results on TS500

In [17]:
with open('./data/ts/ts500.json','r') as f:
    ts500 = json.load(f)


ts500_list = []
for entry in ts500:
    coords = np.array(entry['coords'])
    ts500_list.append(
        {
                    'title':entry['name'],
                    'seq':entry['seq'],
                    'CA':coords[:,1,:],
                    'C':coords[:,2,:],
                    'O':coords[:,3,:],
                    'N':coords[:,0,:]
        }
    )
exp.test_loader = make_cath_loader(CATH(data=ts500_list), 'SimDesign', 8)

In [18]:
exp.test()
print("median: {:.4f}\t mean: {:.4f}\t std: {:.4f}\t min: {:.4f}\t max: {:.4f}".format(exp.method.median_recovery, exp.method.mean_recovery, exp.method.std_recovery, exp.method.min_recovery, exp.method.max_recovery))

100%|██████████| 500/500 [00:17<00:00, 28.06it/s]

Test Perp: 3.4403, Test Rec: 0.6046

Category Unknown Rec: 0.6046

median: 0.6046	 mean: 0.5932	 std: 0.1077	 min: 0.0296	 max: 0.8680
